In [1]:
# Import packages
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

import matplotlib.pyplot as plt
import seaborn as sns

# Defining all our palette colours.
primary_blue = "#496595"
primary_blue2 = "#85a1c1"
primary_blue3 = "#3f4d63"
primary_grey = "#c6ccd8"
primary_black = "#202022"
primary_bgcolor = "#f4f0ea"

# Load and Merge Datasets
In this section we will load the *video game global sales* dataset, and the *video games developer base country* dataset. We will then merge the two datasets to create a single dataset that contains all the information we need to answer the questions we have.

In [2]:
# Import dataset
data_file_path = "datasets/video-game-sales.csv"
companie_region_path = "datasets/video-games-developers.csv"

# Load the data
df = pd.read_csv(data_file_path)
df_companies = pd.read_csv(companie_region_path)

# Merge the two datasets
df = pd.merge(df, df_companies[['Developer', 'Country']], left_on='Publisher', right_on='Developer', how='left')
df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Developer,Country
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,Nintendo,Japan
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,Nintendo,Japan
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82,Nintendo,Japan
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00,Nintendo,Japan
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,Nintendo,Japan


# The Top-selling Game of Each Year

This code snippet is designed to visualize the total video game sales of each year, and contains the top-selling game of each specific year.

In [3]:
# Get the top game in each year
top_games = df.sort_values('Global_Sales', ascending=False).drop_duplicates(['Year'])

# Create the plot
fig = px.bar(top_games, x='Year', y='Global_Sales', color='Year', text='Name', 
             labels={'Year': 'Year', 'Global_Sales': 'Total Sales', 'Name': 'Top Game'}, 
             title='Top Game in Each Year')

fig.update_traces(textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()